In [1]:
# importing libraries
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime
import random

In [2]:
# load data
ml_df = pd.read_csv('data/ML.csv')
ml_tag = pd.read_csv('data/MLtag.csv')
synthentic_df = pd.read_csv('data/Synthetic_Financial_Datasets.csv', nrows=1_00_000)

In [3]:
synthentic_df.drop(['isFlaggedFraud'], axis=1, inplace=True)

In [4]:
synthentic_df['date'] = synthentic_df['step'].apply(lambda x: pd.to_datetime('2019-02-20') + pd.offsets.Hour(x))

In [5]:
mapper = {
    'type': 'typeofaction',
    'nameOrig': 'sourceid',
    'nameDest': 'destinationid',
    'amount': 'amountofmoney',
    'date': 'date',
    'isFraud': 'isfraud',
}

clean_df = synthentic_df.rename(columns = mapper)

In [6]:
clean_df = clean_df[mapper.values()]

In [7]:
clean_df['typeoffraud'] = clean_df['isfraud'].apply(lambda x: f'type{random.randint(1, 2)}' if x == 1 else 'none')

In [8]:
ml_df.typeofaction.replace({
    'transfer': 'TRANSFER',
    'cash-in': 'CASH_IN'
}, inplace=True)

In [9]:
clean_df.head()

,typeofaction,sourceid,destinationid,amountofmoney,date,isfraud,typeoffraud
0,PAYMENT,C1231006815,M1979787155,9839.64,2019-02-20 01:00:00,0,none
1,PAYMENT,C1666544295,M2044282225,1864.28,2019-02-20 01:00:00,0,none
2,TRANSFER,C1305486145,C553264065,181.00,2019-02-20 01:00:00,1,type1
3,CASH_OUT,C840083671,C38997010,181.00,2019-02-20 01:00:00,1,type1
4,PAYMENT,C2048537720,M1230701703,11668.14,2019-02-20 01:00:00,0,none


In [10]:
ml_df.head()

,typeofaction,sourceid,destinationid,amountofmoney,date,isfraud,typeoffraud
0,CASH_IN,30105,28942,494528,2019-07-19 14:40:00,1,type1
1,CASH_IN,30105,8692,494528,2019-05-17 14:57:00,1,type1
2,CASH_IN,30105,60094,494528,2019-07-20 13:20:00,1,type1
3,CASH_IN,30105,20575,494528,2019-07-03 14:15:00,1,type1
4,CASH_IN,30105,45938,494528,2019-05-26 10:40:00,1,type1


In [11]:
main_df = pd.concat([clean_df, ml_df], axis=0)

In [12]:
main_df.head()

,typeofaction,sourceid,destinationid,amountofmoney,date,isfraud,typeoffraud
0,PAYMENT,C1231006815,M1979787155,9839.64,2019-02-20 01:00:00,0,none
1,PAYMENT,C1666544295,M2044282225,1864.28,2019-02-20 01:00:00,0,none
2,TRANSFER,C1305486145,C553264065,181.00,2019-02-20 01:00:00,1,type1
3,CASH_OUT,C840083671,C38997010,181.00,2019-02-20 01:00:00,1,type1
4,PAYMENT,C2048537720,M1230701703,11668.14,2019-02-20 01:00:00,0,none


In [13]:
def one_hot(df, key='typeofaction'):
    return pd.concat([
            main_df,
            pd.get_dummies(main_df[key], prefix=key, drop_first=True)
        ], axis=1)

In [14]:
main_df = one_hot(main_df)

In [15]:
main_df.drop('typeofaction', axis=1, inplace=True)
main_df.head()

,sourceid,destinationid,amountofmoney,date,isfraud,typeoffraud,typeofaction_CASH_OUT,typeofaction_DEBIT,typeofaction_PAYMENT,typeofaction_TRANSFER
0,C1231006815,M1979787155,9839.64,2019-02-20 01:00:00,0,none,0,0,1,0
1,C1666544295,M2044282225,1864.28,2019-02-20 01:00:00,0,none,0,0,1,0
2,C1305486145,C553264065,181.00,2019-02-20 01:00:00,1,type1,0,0,0,1
3,C840083671,C38997010,181.00,2019-02-20 01:00:00,1,type1,1,0,0,0
4,C2048537720,M1230701703,11668.14,2019-02-20 01:00:00,0,none,0,0,1,0


In [16]:
main_df = main_df.reset_index(drop=True)
main_df[['sourceid', 'destinationid']] = main_df[['sourceid', 'destinationid']].astype(str)

In [17]:
main_df['date'] = pd.to_datetime(main_df['date'])

In [18]:
main_df.to_pickle("main_df.pkl")

## Start from here if you have a pickled file

In [19]:
main_df = pd.read_pickle('main_df.pkl')

In [20]:
main_df.drop('typeoffraud', axis=1, inplace=True)

In [21]:
# main_df = one_hot(main_df, 'sourceid')
# main_df = one_hot(main_df, 'destinationid')

# drop col
# main_df.drop(['destinationid', 'sourceid'], axis=1, inplace=True)

In [22]:
main_df['prefix_source'] = main_df['sourceid'].str[:1]
main_df['prefix_destination'] = main_df['destinationid'].str[:1]

In [23]:
main_df.head()

,sourceid,destinationid,amountofmoney,date,isfraud,typeofaction_CASH_OUT,typeofaction_DEBIT,typeofaction_PAYMENT,typeofaction_TRANSFER,prefix_source,prefix_destination
0,C1231006815,M1979787155,9839.64,2019-02-20 01:00:00,0,0,0,1,0,C,M
1,C1666544295,M2044282225,1864.28,2019-02-20 01:00:00,0,0,0,1,0,C,M
2,C1305486145,C553264065,181.00,2019-02-20 01:00:00,1,0,0,0,1,C,C
3,C840083671,C38997010,181.00,2019-02-20 01:00:00,1,1,0,0,0,C,C
4,C2048537720,M1230701703,11668.14,2019-02-20 01:00:00,0,0,0,1,0,C,M


In [24]:
def create_datetime_features(df, col, time_params_reqd = False, remove_main_date=False, prefix='temp'):
    """
    Function to extract features from the datetime column and return the df with the updated values

    Example:
    create_datetime_features(portfoliostats[['BookedDate']], 'BookedDate', prefix='bookedDate', remove_main_date=True)
    """
    if df[col].dtype != 'datetime64[ns]':
        df[col] = pd.to_datetime(df[col]) 
        
    df[f'{prefix}_year'] = df[col].dt.year
    df[f'{prefix}_month'] = df[col].dt.month
    df[f'{prefix}_day'] = df[col].dt.day
    df[f'{prefix}_week'] = df[col].dt.week
    df[f'{prefix}_dayofweek'] = df[col].dt.dayofweek
    df[f'{prefix}_isweekend'] = (df[col].dt.dayofweek >=5).astype(int)
    df[f'{prefix}_isWeekday'] = (df[col].dt.dayofweek < 5).astype(int)
    df[f'{prefix}_dayofyear'] = df[col].dt.dayofyear
    df[f'{prefix}_quarter'] = df[col].dt.quarter
    df[f'{prefix}_isleapyear'] = df[col].dt.is_leap_year.astype(int)
    df[f'{prefix}_monthDifferenceFromToday'] = (datetime.datetime.today() - df[col]).dt.days//30
    df[f'{prefix}_dayDifferenceFromToday'] = (datetime.datetime.today() - df[col]).dt.days

    if time_params_reqd:
        df[f'{prefix}_hour'] = df[col].dt.hour
        df[f'{prefix}_minute'] = df[col].dt.minute

    if remove_main_date:
        df.drop([col], axis=1, inplace=True)
    
    return df

In [25]:
main_df = main_df.sort_values('date')

In [26]:
main_df['date_lag'] = main_df.groupby('sourceid')['date'].shift(-1)
main_df['date_lag'].fillna(pd.to_datetime('2019-02-13'), inplace=True)
main_df['days_since_last_transaction'] = (main_df['date'] - main_df['date_lag']).dt.days
main_df['hours_since_last_transaction'] = (main_df['date'] - main_df['date_lag']).dt.components['hours']
main_df['mins_since_last_transaction'] = (main_df['date'] - main_df['date_lag']).dt.components['minutes']

In [27]:
main_df = create_datetime_features(main_df, 'date', True, False)

/Users/pansul/Documents/time-series/time-series-final-project/venv/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [28]:
main_df.head()

,sourceid,destinationid,amountofmoney,date,isfraud,typeofaction_CASH_OUT,typeofaction_DEBIT,typeofaction_PAYMENT,typeofaction_TRANSFER,prefix_source,...,temp_dayofweek,temp_isweekend,temp_isWeekday,temp_dayofyear,temp_quarter,temp_isleapyear,temp_monthDifferenceFromToday,temp_dayDifferenceFromToday,temp_hour,temp_minute
0,C1231006815,M1979787155,9839.64,2019-02-20 01:00:00,0,0,0,1,0,C,...,2,0,1,51,1,0,20,618,1,0
1801,C1562950869,M2021835850,5157.05,2019-02-20 01:00:00,0,0,0,1,0,C,...,2,0,1,51,1,0,20,618,1,0
1802,C845388562,M550572371,5746.44,2019-02-20 01:00:00,0,0,0,1,0,C,...,2,0,1,51,1,0,20,618,1,0
1803,C948424584,M1447685190,5607.36,2019-02-20 01:00:00,0,0,0,1,0,C,...,2,0,1,51,1,0,20,618,1,0
1804,C2027701910,M1345293143,6360.79,2019-02-20 01:00:00,0,0,0,1,0,C,...,2,0,1,51,1,0,20,618,1,0


In [29]:
main_df.drop(['destinationid', 'sourceid', 'date'], axis=1, inplace=True, errors='ignore')

In [30]:
pd.options.display.max_columns = None

In [31]:
main_df.head()

,amountofmoney,isfraud,typeofaction_CASH_OUT,typeofaction_DEBIT,typeofaction_PAYMENT,typeofaction_TRANSFER,prefix_source,prefix_destination,date_lag,days_since_last_transaction,hours_since_last_transaction,mins_since_last_transaction,temp_year,temp_month,temp_day,temp_week,temp_dayofweek,temp_isweekend,temp_isWeekday,temp_dayofyear,temp_quarter,temp_isleapyear,temp_monthDifferenceFromToday,temp_dayDifferenceFromToday,temp_hour,temp_minute
0,9839.64,0,0,0,1,0,C,M,2019-02-13,7,1,0,2019,2,20,8,2,0,1,51,1,0,20,618,1,0
1801,5157.05,0,0,0,1,0,C,M,2019-02-13,7,1,0,2019,2,20,8,2,0,1,51,1,0,20,618,1,0
1802,5746.44,0,0,0,1,0,C,M,2019-02-13,7,1,0,2019,2,20,8,2,0,1,51,1,0,20,618,1,0
1803,5607.36,0,0,0,1,0,C,M,2019-02-13,7,1,0,2019,2,20,8,2,0,1,51,1,0,20,618,1,0
1804,6360.79,0,0,0,1,0,C,M,2019-02-13,7,1,0,2019,2,20,8,2,0,1,51,1,0,20,618,1,0
